In [1]:
# Import modules
import pandas as pd
import scanpy as sc
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import scipy.sparse as sp
from tqdm.auto import tqdm
from scipy.stats.contingency import crosstab
import seaborn as sns
import matplotlib.pyplot as plt

/home/inf-39-2024/.conda/envs/scseq/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
adata = sc.read_h5ad('8_Classifiers/Data/Input/scanpy_cp_scAtlas.h5ad')
adata

AnnData object with n_obs × n_vars = 52997 × 45454
    obs: 'nCount_RNA', 'nFeature_RNA', 'harm_study', 'harm_healthy.tissue', 'harm_tumor.site', 'harm_sample.type', 'harm_condition', 'harm_tumor.type', 'harm_cd45pos', 'harm_healthy.pat', 'percent.mt', 'ratio_nCount_nFeature', 'batch', 'X_scvi_batch', 'X_scvi_labels', 'X_scvi_local_l_mean', 'X_scvi_local_l_var', 'leiden_0.2', 'leiden_0.4', 'leiden_0.6', 'leiden_0.8', 'leiden_1', 'leiden_1.2', 'leiden_1.4', 'author_first_cell_type', 'author_cell_type', 'cnv_score', 'organism_ontology_term_id', 'donor_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id', 'cell_type_ontology_term_id', 'suspension_type', 'assay_ontology_term_id', 'is_primary_data', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'doublet_scores', 'predicted_doublets

In [ ]:
# Define mapping from fine cell types to general groups
cell_type_groups = {
    # Monocytes
    'Mono_CD14_FOS+': 'Monocytes',
    'Mono_CD14_FOS-': 'Monocytes',
    'Mono_CD16': 'Monocytes',
    'Mono_IL1B': 'Monocytes',
    'MonoInter_CLEC10A': 'Monocytes',
    'MonoInter_CXCL10': 'Monocytes',

    # Macrophages
    'Mac_Rec': 'Macrophages',
    'Mac_Hypo': 'Macrophages',
    'Mac_IFN': 'Macrophages',
    'Mac_AgPres': 'Macrophages',
    'Mac_Angio': 'Macrophages',
    'Mac_LA': 'Macrophages',
    'Mac_Alv-like': 'Macrophages',
    'RTM_IM': 'Macrophages',
    'RTM_IFN': 'Macrophages',
    'RTM_LA': 'Macrophages',
    'RTM-like_MT': 'Macrophages',

    # Dendritic Cells
    'cDC1_CLEC9A': 'DCs',
    'cDC2_CD207': 'DCs',
    'cDC2_AREG': 'DCs',
    'cDC2_FCER1A': 'DCs',
    'cDC3_CD14': 'DCs',
    'cDC4_FCGR3A': 'DCs',
    'cDC_CXCL8': 'DCs',
    'cDC_LAMP3': 'DCs',

    # Rest
    'pDC': 'Rest',}

# Default group for any cell type not explicitly mapped
default_group = 'Rest'

# Apply mapping to create a new column in adata.obs
adata.obs['cell_type_group'] = adata.obs['author_cell_type'].map(cell_type_groups).fillna(default_group)


In [ ]:
adata.obs['cell_type_group'].unique()

array(['Macrophages', 'Monocytes', 'DCs', 'Rest'], dtype=object)

In [ ]:
adata.write_h5ad('8_Classifiers/Data/Input/scAtlas_grouped.h5ad')